In [19]:
! wget 'https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1'

--2021-07-25 21:15:36--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin?download=1’

SO_vectors_200.bin? 100%[===================>]   1.35G  25.5MB/s    in 54s     

2021-07-25 21:16:32 (25.7 MB/s) - ‘SO_vectors_200.bin?download=1’ saved [1453905423/1453905423]



In [4]:
! unzip "/content/drive/MyDrive/CVs.zip"

Archive:  /content/drive/MyDrive/CVs.zip
   creating: CVs/
   creating: CVs/CVs1/
  inflating: CVs/CVs1/A1_PWC_Joseph Leung_Audit Senior.pdf  
  inflating: CVs/CVs1/A2_China Renaissance_QUIN LAM_Fund Acc_PE.doc  
  inflating: CVs/CVs1/A3_Augentius_Dianie Samad_Senior Accountant.docx  
  inflating: CVs/CVs1/A4_Intertrust_Tracy Ho_Fund Acc_PE.docx  
  inflating: CVs/CVs1/A5_Deloitte_Henry Chu_Audit Senior.docx  
  inflating: CVs/CVs1/And1_Abdallah Naguib - Software Developer.pdf  
  inflating: CVs/CVs1/D1_nlp-data-scientist-resume.pdf  
  inflating: CVs/CVs1/D2_Senior Data Scientist Resume.pdf  
  inflating: CVs/CVs1/D3_data-scientist-resume-sample.pdf  
  inflating: CVs/CVs1/D4_Entry Level Data Scientist Resume.pdf  
  inflating: CVs/CVs1/D5_data-scientist.pdf  
  inflating: CVs/CVs1/G1_game-developer.pdf  
  inflating: CVs/CVs1/G2_game-developer.pdf  
  inflating: CVs/CVs1/G3_game-developer.pdf  
  inflating: CVs/CVs1/G4_game-developer.pdf  
  inflating: CVs/CVs1/G5_game-developer.pdf 

In [1]:
! pip install textract --upgrade

In [21]:
from gensim.models.doc2vec import TaggedDocument , Doc2Vec
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.porter import PorterStemmer
from gensim.utils import simple_preprocess
from sklearn.datasets import load_files
from gensim.models import KeyedVectors
from google.colab import files, drive
from os.path import isfile, join
from os import listdir
import multiprocessing
import pandas as pd
import numpy as np 
import nltk as nl
import traceback
import textract
import logging
import pickle
import csv
nl.download('stopwords')
porter_stemmer = PorterStemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [40]:
def cosine(A, B):   return np.dot(A,B)/(np.linalg.norm(A)*np.linalg.norm(B))

def show_dict(dictionary):
  cnt = 0
  for i in dictionary:
    cnt += 1
    print('Index :', cnt, i)

def read_All_CV(filename):
    text = textract.process(filename)
    return text.decode('utf-8')

def load_data(path): 
    with open(path) as csv_file:
      read_csv=csv.reader(csv_file,delimiter=',')
      next(read_csv,None) # skip the headers
      df = pd.DataFrame(read_csv)
    return df

def preprocess(docs):
    #stop words removal
    docs = [remove_stopwords(doc) for doc in docs]

    #tokenization
    docs = [simple_preprocess(doc) for doc in docs] 

    #stemming
    docs = [[porter_stemmer.stem(word) for word in doc] for doc in docs]
    return docs

def vectorize(model, words) -> np.ndarray:
        
        word_vecs = []
        for word in words:
            try:
                vec = model[word]
                word_vecs.append(vec)
            except KeyError:
                # Ignore, if the word doesn't exist in the vocabulary
                pass

        # Assuming that document vector is the mean of all the word vectors
        # PS: There are other & better ways to do it.
        vector = np.mean(word_vecs, axis=0)
        return vector

def calculate_w2v_similarity(word2vec_model, dircvs, source_doc, target_docs=None, threshold=-1e9):
        """Calculates & returns similarity scores between given source document & all
        the target documents."""
        if not target_docs:
            return []

        #Vectorize using Word2Vec
        source_vec = vectorize(word2vec_model,source_doc)
        results = []
        for i in range(len(target_docs)):
            #Vectorize using Word2Vec
            target_vec = vectorize(word2vec_model,target_docs[i])
            #calculating similarity using cosine similarity
            sim_score = cosine(source_vec, target_vec)
            if sim_score > threshold:
                results.append({"score": sim_score, "doc": dircvs[i]})
            # Sort results by score in desc order
            results.sort(key=lambda k: k["score"], reverse=True)
        return results

def calculate_similarity(doc2vec_model, dircvs, source_doc, target_docs=None, threshold=-1e9):
        """Calculates & returns similarity scores between given source document & all
        the target documents."""
        if not target_docs:
            return []

        #Vectorize using Doc2Vec
        source_vec = doc2vec_model.infer_vector(source_doc)
        results = []
        for i in range(len(target_docs)):
            #Vectorize using Doc2Vec
            target_vec =  doc2vec_model.infer_vector(target_docs[i])
            #calculating similarity using cosine similarity
            sim_score = cosine(source_vec, target_vec)
            if sim_score > threshold:
                results.append({"score": sim_score, "doc": dircvs[i]})
            # Sort results by score in desc order
            results.sort(key=lambda k: k["score"], reverse=True)
        return results

def read_files_in_dir(directory):
  target_docs = []
  dircvs = [join(directory, f) for f in listdir(directory) if isfile(join(directory, f))]
  for cv in dircvs:
    try:
      text = read_All_CV(cv)
      target_docs.append(text)
    except Exception:
      logging.error(traceback.format_exc())
      print(cv,'\n')
  return dircvs ,target_docs

def show_parameters(model):
  if (model.dm): 
    _type_ = 'PV-DM' 
  else:
    _type_ = 'PV-DBOW'
  print('Type :',_type_)
  print('Dimensions :',model.vector_size)
  print('Window :',model.window)
  print('Min Word Count :',model.vocabulary.min_count)
  print('Alpha :',model.alpha)
  print('Min Alpha :',model.min_alpha)
  print('Epochs :',model.epochs)
  return

def intialize_test_data():
  JD_titles , JD_contents = read_files_in_dir('/content/JDs')
  JD_contents = preprocess(JD_contents)
  dircvs1 , CVs1 = read_files_in_dir('/content/CVs/CVs1')
  dircvs2 , CVs2 = read_files_in_dir('/content/CVs/CVs2')
  dircvs3 , CVs3 = read_files_in_dir('/content/CVs/CVs3')
  dircvs4 , CVs4 = read_files_in_dir('/content/CVs/CVs4')
  CVs1 = preprocess(CVs1)
  CVs2 = preprocess(CVs2)
  CVs3 = preprocess(CVs3)
  CVs4 = preprocess(CVs4)
  return JD_titles, [dircvs1, dircvs2, dircvs3, dircvs4] , JD_contents, [CVs1, CVs2, CVs3, CVs4]

def test_w2v_model(model, JD_titles, dircvs, JD_contents, CVs):
  test_cnt = 0
  for i in range(len(JD_contents)):
    for j in range(len(CVs)):
      test_cnt += 1
      print('Test Case',test_cnt,':')
      print('Job Description :', JD_titles[i])
      print('With Set :', dircvs[j])
      show_dict(calculate_w2v_similarity(model,dircvs[j],JD_contents[i],CVs[j]))
      print('CVs no. :',len(CVs[j]))
      print('='*500)
  return

def test_model(model, JD_titles, dircvs, JD_contents, CVs):
  test_cnt = 0
  for i in range(len(JD_contents)):
    for j in range(len(CVs)):
      test_cnt += 1
      print('Test Case',test_cnt,':')
      print('Job Description :', JD_titles[i])
      print('With Set :', dircvs[j])
      show_dict(calculate_similarity(model,dircvs[j],JD_contents[i],CVs[j]))
      print('CVs no. :',len(CVs[j]))
      print('='*500)
  return

In [20]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
model = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/SO_vectors_200.bin', binary=True)
print('Dimensions :',model.vector_size)

Dimensions : 200


In [46]:
model2 =  Doc2Vec.load("/content/drive/MyDrive/doc2vecmodel100dm1")
show_parameters(model2)

Type : PV-DM
Dimensions : 100
Window : 3
Min Word Count : 30
Alpha : 0.025
Min Alpha : 0.00025
Epochs : 70


In [4]:
model3 = Doc2Vec.load("/content/drive/MyDrive/doc2vec_100_1_40_5_StkOvFl_posts")
show_parameters(model3)

Type : PV-DM
Dimensions : 300
Window : 5
Min Word Count : 1
Alpha : 0.025
Min Alpha : 0.0001
Epochs : 100


In [5]:
model4 = Doc2Vec.load("/content/drive/MyDrive/doc2vec_100_1_40_5_1K_CVs")
show_parameters(model4)

Type : PV-DM
Dimensions : 300
Window : 5
Min Word Count : 1
Alpha : 0.025
Min Alpha : 0.0001
Epochs : 100


In [6]:
model5 = Doc2Vec.load("/content/drive/MyDrive/doc2vec_100_1_40_5_2.5K_CVs")
show_parameters(model5)

Type : PV-DM
Dimensions : 300
Window : 5
Min Word Count : 1
Alpha : 0.025
Min Alpha : 0.0001
Epochs : 100


In [7]:
model6 = Doc2Vec.load("/content/drive/MyDrive/doc2vec_100_3_40_5_2.5K_CVs")
show_parameters(model6)

Type : PV-DM
Dimensions : 300
Window : 5
Min Word Count : 3
Alpha : 0.025
Min Alpha : 0.0001
Epochs : 100


In [8]:
model7 = Doc2Vec.load("/content/drive/MyDrive/doc2vec_100_7_40_5_2.5K_CVs")
show_parameters(model7)

Type : PV-DM
Dimensions : 300
Window : 5
Min Word Count : 7
Alpha : 0.025
Min Alpha : 0.0001
Epochs : 100


In [9]:
model8 = Doc2Vec.load("/content/drive/MyDrive/doc2vec_100_1_40_5_7K_CVs")
show_parameters(model8)

Type : PV-DM
Dimensions : 300
Window : 5
Min Word Count : 1
Alpha : 0.025
Min Alpha : 0.0001
Epochs : 100


In [10]:
model9 = Doc2Vec.load("/content/drive/MyDrive/doc2vec_100_3_40_5_7K_CVs")
show_parameters(model9)

Type : PV-DM
Dimensions : 300
Window : 5
Min Word Count : 3
Alpha : 0.025
Min Alpha : 0.0001
Epochs : 100


In [11]:
JD_titles, dircvs, JD_contents, CVs = intialize_test_data()

In [45]:
test_w2v_model(model, JD_titles, dircvs, JD_contents, CVs)

Test Case 1 :
Job Description : /content/JDs/GameDev.docx
With Set : ['/content/CVs/CVs1/D5_data-scientist.pdf', '/content/CVs/CVs1/And1_Abdallah Naguib - Software Developer.pdf', '/content/CVs/CVs1/P2_penetration-tester.pdf', '/content/CVs/CVs1/S4_Scrum Master Resume.pdf', '/content/CVs/CVs1/G2_game-developer.pdf', '/content/CVs/CVs1/S1_Agile Scrum Master Resume.pdf', '/content/CVs/CVs1/R2_ruby-on-rails-developer.pdf', '/content/CVs/CVs1/D4_Entry Level Data Scientist Resume.pdf', '/content/CVs/CVs1/D1_nlp-data-scientist-resume.pdf', '/content/CVs/CVs1/G4_game-developer.pdf', '/content/CVs/CVs1/S2_CSM Scrum Master Resume Resume.pdf', '/content/CVs/CVs1/S5_Senior Scrum Master Resume.pdf', '/content/CVs/CVs1/S3_Entry Level Scrum Master Resume.pdf', '/content/CVs/CVs1/A4_Intertrust_Tracy Ho_Fund Acc_PE.docx', '/content/CVs/CVs1/J3_senior-java-developer-resume-example.pdf', '/content/CVs/CVs1/A5_Deloitte_Henry Chu_Audit Senior.docx', '/content/CVs/CVs1/D3_data-scientist-resume-sample.pdf',

In [47]:
test_model(model2, JD_titles, dircvs, JD_contents, CVs)

Test Case 1 :
Job Description : /content/JDs/GameDev.docx
With Set : ['/content/CVs/CVs1/D5_data-scientist.pdf', '/content/CVs/CVs1/And1_Abdallah Naguib - Software Developer.pdf', '/content/CVs/CVs1/P2_penetration-tester.pdf', '/content/CVs/CVs1/S4_Scrum Master Resume.pdf', '/content/CVs/CVs1/G2_game-developer.pdf', '/content/CVs/CVs1/S1_Agile Scrum Master Resume.pdf', '/content/CVs/CVs1/R2_ruby-on-rails-developer.pdf', '/content/CVs/CVs1/D4_Entry Level Data Scientist Resume.pdf', '/content/CVs/CVs1/D1_nlp-data-scientist-resume.pdf', '/content/CVs/CVs1/G4_game-developer.pdf', '/content/CVs/CVs1/S2_CSM Scrum Master Resume Resume.pdf', '/content/CVs/CVs1/S5_Senior Scrum Master Resume.pdf', '/content/CVs/CVs1/S3_Entry Level Scrum Master Resume.pdf', '/content/CVs/CVs1/A4_Intertrust_Tracy Ho_Fund Acc_PE.docx', '/content/CVs/CVs1/J3_senior-java-developer-resume-example.pdf', '/content/CVs/CVs1/A5_Deloitte_Henry Chu_Audit Senior.docx', '/content/CVs/CVs1/D3_data-scientist-resume-sample.pdf',

In [12]:
test_model(model3, JD_titles, dircvs, JD_contents, CVs)

Test Case 1 :
Job Description : /content/JDs/GameDev.docx
With Set : ['/content/CVs/CVs1/D5_data-scientist.pdf', '/content/CVs/CVs1/And1_Abdallah Naguib - Software Developer.pdf', '/content/CVs/CVs1/P2_penetration-tester.pdf', '/content/CVs/CVs1/S4_Scrum Master Resume.pdf', '/content/CVs/CVs1/G2_game-developer.pdf', '/content/CVs/CVs1/S1_Agile Scrum Master Resume.pdf', '/content/CVs/CVs1/R2_ruby-on-rails-developer.pdf', '/content/CVs/CVs1/D4_Entry Level Data Scientist Resume.pdf', '/content/CVs/CVs1/D1_nlp-data-scientist-resume.pdf', '/content/CVs/CVs1/G4_game-developer.pdf', '/content/CVs/CVs1/S2_CSM Scrum Master Resume Resume.pdf', '/content/CVs/CVs1/S5_Senior Scrum Master Resume.pdf', '/content/CVs/CVs1/S3_Entry Level Scrum Master Resume.pdf', '/content/CVs/CVs1/A4_Intertrust_Tracy Ho_Fund Acc_PE.docx', '/content/CVs/CVs1/J3_senior-java-developer-resume-example.pdf', '/content/CVs/CVs1/A5_Deloitte_Henry Chu_Audit Senior.docx', '/content/CVs/CVs1/D3_data-scientist-resume-sample.pdf',

In [13]:
test_model(model4, JD_titles, dircvs, JD_contents, CVs)

Test Case 1 :
Job Description : /content/JDs/GameDev.docx
With Set : ['/content/CVs/CVs1/D5_data-scientist.pdf', '/content/CVs/CVs1/And1_Abdallah Naguib - Software Developer.pdf', '/content/CVs/CVs1/P2_penetration-tester.pdf', '/content/CVs/CVs1/S4_Scrum Master Resume.pdf', '/content/CVs/CVs1/G2_game-developer.pdf', '/content/CVs/CVs1/S1_Agile Scrum Master Resume.pdf', '/content/CVs/CVs1/R2_ruby-on-rails-developer.pdf', '/content/CVs/CVs1/D4_Entry Level Data Scientist Resume.pdf', '/content/CVs/CVs1/D1_nlp-data-scientist-resume.pdf', '/content/CVs/CVs1/G4_game-developer.pdf', '/content/CVs/CVs1/S2_CSM Scrum Master Resume Resume.pdf', '/content/CVs/CVs1/S5_Senior Scrum Master Resume.pdf', '/content/CVs/CVs1/S3_Entry Level Scrum Master Resume.pdf', '/content/CVs/CVs1/A4_Intertrust_Tracy Ho_Fund Acc_PE.docx', '/content/CVs/CVs1/J3_senior-java-developer-resume-example.pdf', '/content/CVs/CVs1/A5_Deloitte_Henry Chu_Audit Senior.docx', '/content/CVs/CVs1/D3_data-scientist-resume-sample.pdf',

In [14]:
test_model(model5, JD_titles, dircvs, JD_contents, CVs)

Test Case 1 :
Job Description : /content/JDs/GameDev.docx
With Set : ['/content/CVs/CVs1/D5_data-scientist.pdf', '/content/CVs/CVs1/And1_Abdallah Naguib - Software Developer.pdf', '/content/CVs/CVs1/P2_penetration-tester.pdf', '/content/CVs/CVs1/S4_Scrum Master Resume.pdf', '/content/CVs/CVs1/G2_game-developer.pdf', '/content/CVs/CVs1/S1_Agile Scrum Master Resume.pdf', '/content/CVs/CVs1/R2_ruby-on-rails-developer.pdf', '/content/CVs/CVs1/D4_Entry Level Data Scientist Resume.pdf', '/content/CVs/CVs1/D1_nlp-data-scientist-resume.pdf', '/content/CVs/CVs1/G4_game-developer.pdf', '/content/CVs/CVs1/S2_CSM Scrum Master Resume Resume.pdf', '/content/CVs/CVs1/S5_Senior Scrum Master Resume.pdf', '/content/CVs/CVs1/S3_Entry Level Scrum Master Resume.pdf', '/content/CVs/CVs1/A4_Intertrust_Tracy Ho_Fund Acc_PE.docx', '/content/CVs/CVs1/J3_senior-java-developer-resume-example.pdf', '/content/CVs/CVs1/A5_Deloitte_Henry Chu_Audit Senior.docx', '/content/CVs/CVs1/D3_data-scientist-resume-sample.pdf',

In [15]:
test_model(model6, JD_titles, dircvs, JD_contents, CVs)

Test Case 1 :
Job Description : /content/JDs/GameDev.docx
With Set : ['/content/CVs/CVs1/D5_data-scientist.pdf', '/content/CVs/CVs1/And1_Abdallah Naguib - Software Developer.pdf', '/content/CVs/CVs1/P2_penetration-tester.pdf', '/content/CVs/CVs1/S4_Scrum Master Resume.pdf', '/content/CVs/CVs1/G2_game-developer.pdf', '/content/CVs/CVs1/S1_Agile Scrum Master Resume.pdf', '/content/CVs/CVs1/R2_ruby-on-rails-developer.pdf', '/content/CVs/CVs1/D4_Entry Level Data Scientist Resume.pdf', '/content/CVs/CVs1/D1_nlp-data-scientist-resume.pdf', '/content/CVs/CVs1/G4_game-developer.pdf', '/content/CVs/CVs1/S2_CSM Scrum Master Resume Resume.pdf', '/content/CVs/CVs1/S5_Senior Scrum Master Resume.pdf', '/content/CVs/CVs1/S3_Entry Level Scrum Master Resume.pdf', '/content/CVs/CVs1/A4_Intertrust_Tracy Ho_Fund Acc_PE.docx', '/content/CVs/CVs1/J3_senior-java-developer-resume-example.pdf', '/content/CVs/CVs1/A5_Deloitte_Henry Chu_Audit Senior.docx', '/content/CVs/CVs1/D3_data-scientist-resume-sample.pdf',

In [16]:
test_model(model7, JD_titles, dircvs, JD_contents, CVs)

Test Case 1 :
Job Description : /content/JDs/GameDev.docx
With Set : ['/content/CVs/CVs1/D5_data-scientist.pdf', '/content/CVs/CVs1/And1_Abdallah Naguib - Software Developer.pdf', '/content/CVs/CVs1/P2_penetration-tester.pdf', '/content/CVs/CVs1/S4_Scrum Master Resume.pdf', '/content/CVs/CVs1/G2_game-developer.pdf', '/content/CVs/CVs1/S1_Agile Scrum Master Resume.pdf', '/content/CVs/CVs1/R2_ruby-on-rails-developer.pdf', '/content/CVs/CVs1/D4_Entry Level Data Scientist Resume.pdf', '/content/CVs/CVs1/D1_nlp-data-scientist-resume.pdf', '/content/CVs/CVs1/G4_game-developer.pdf', '/content/CVs/CVs1/S2_CSM Scrum Master Resume Resume.pdf', '/content/CVs/CVs1/S5_Senior Scrum Master Resume.pdf', '/content/CVs/CVs1/S3_Entry Level Scrum Master Resume.pdf', '/content/CVs/CVs1/A4_Intertrust_Tracy Ho_Fund Acc_PE.docx', '/content/CVs/CVs1/J3_senior-java-developer-resume-example.pdf', '/content/CVs/CVs1/A5_Deloitte_Henry Chu_Audit Senior.docx', '/content/CVs/CVs1/D3_data-scientist-resume-sample.pdf',

In [17]:
test_model(model8, JD_titles, dircvs, JD_contents, CVs)

Test Case 1 :
Job Description : /content/JDs/GameDev.docx
With Set : ['/content/CVs/CVs1/D5_data-scientist.pdf', '/content/CVs/CVs1/And1_Abdallah Naguib - Software Developer.pdf', '/content/CVs/CVs1/P2_penetration-tester.pdf', '/content/CVs/CVs1/S4_Scrum Master Resume.pdf', '/content/CVs/CVs1/G2_game-developer.pdf', '/content/CVs/CVs1/S1_Agile Scrum Master Resume.pdf', '/content/CVs/CVs1/R2_ruby-on-rails-developer.pdf', '/content/CVs/CVs1/D4_Entry Level Data Scientist Resume.pdf', '/content/CVs/CVs1/D1_nlp-data-scientist-resume.pdf', '/content/CVs/CVs1/G4_game-developer.pdf', '/content/CVs/CVs1/S2_CSM Scrum Master Resume Resume.pdf', '/content/CVs/CVs1/S5_Senior Scrum Master Resume.pdf', '/content/CVs/CVs1/S3_Entry Level Scrum Master Resume.pdf', '/content/CVs/CVs1/A4_Intertrust_Tracy Ho_Fund Acc_PE.docx', '/content/CVs/CVs1/J3_senior-java-developer-resume-example.pdf', '/content/CVs/CVs1/A5_Deloitte_Henry Chu_Audit Senior.docx', '/content/CVs/CVs1/D3_data-scientist-resume-sample.pdf',

In [18]:
test_model(model9, JD_titles, dircvs, JD_contents, CVs)

Test Case 1 :
Job Description : /content/JDs/GameDev.docx
With Set : ['/content/CVs/CVs1/D5_data-scientist.pdf', '/content/CVs/CVs1/And1_Abdallah Naguib - Software Developer.pdf', '/content/CVs/CVs1/P2_penetration-tester.pdf', '/content/CVs/CVs1/S4_Scrum Master Resume.pdf', '/content/CVs/CVs1/G2_game-developer.pdf', '/content/CVs/CVs1/S1_Agile Scrum Master Resume.pdf', '/content/CVs/CVs1/R2_ruby-on-rails-developer.pdf', '/content/CVs/CVs1/D4_Entry Level Data Scientist Resume.pdf', '/content/CVs/CVs1/D1_nlp-data-scientist-resume.pdf', '/content/CVs/CVs1/G4_game-developer.pdf', '/content/CVs/CVs1/S2_CSM Scrum Master Resume Resume.pdf', '/content/CVs/CVs1/S5_Senior Scrum Master Resume.pdf', '/content/CVs/CVs1/S3_Entry Level Scrum Master Resume.pdf', '/content/CVs/CVs1/A4_Intertrust_Tracy Ho_Fund Acc_PE.docx', '/content/CVs/CVs1/J3_senior-java-developer-resume-example.pdf', '/content/CVs/CVs1/A5_Deloitte_Henry Chu_Audit Senior.docx', '/content/CVs/CVs1/D3_data-scientist-resume-sample.pdf',

In [44]:
import gc
gc.collect()

435233